# Medical acts

Generate a realistic distribution of cares deliveries

In [3]:
import pandas as pd
import numpy as np

### Extraction d'information sur les actes médicaux depuis le OpenDAMIR

On charge la base de l'OpenDamir pour récupérer des informations sur les actes médicaux. Ces actes sont repérés par les codes PRS_NAT de type 13XX 

On parle ici uniquement des actes type CCAM. On écarte en particulier les soins infirmiers, etc. 

On écarte également de nouveau tous les soins dentaires !

In [4]:
#Ouverture du jeu de données OpenDAMIR
damir=pd.read_csv("../data/A201604.csv", sep=';', usecols=['BEN_SEX_COD','AGE_BEN_SNDS','BEN_RES_REG','PSE_ACT_CAT','PSE_SPE_SNDS', 'PRS_NAT', "PRS_ACT_QTE"])
damir.rename(columns={"BEN_RES_REG":"RR","BEN_SEX_COD":"sex","AGE_BEN_SNDS":"age","PRS_NAT":"prs_nat","PSE_ACT_CAT":"pse_cat","PSE_SPE_SNDS":"exe_spe", "PRS_ACT_QTE":"act_dnb"}, inplace=True)
damir_dnb=damir.groupby(["RR","sex","age","prs_nat","pse_cat","exe_spe"]).agg({"act_dnb":'sum'}).reset_index()
damir_dnb.columns = damir_dnb.columns.get_level_values(0)

In [5]:
#Sélection uniquement des codes de ANT_PRS de type 13XX : actes médicaux

acts=damir_dnb[(damir_dnb["prs_nat"]<1400) &(damir_dnb["prs_nat"]>=1300) & (damir_dnb["act_dnb"]>=0)]

In [7]:
#on regroupe les actes délivrées par des médecins ou des établissements (code pse_cat 0 et 1)
acts=acts.groupby(["RR","sex","age","prs_nat","exe_spe"]).agg({"act_dnb":"sum"}).reset_index()
acts.head()

,RR,sex,age,prs_nat,exe_spe,act_dnb
0,5,1,0,1311,1,2
1,5,1,0,1312,1,65
2,5,1,0,1312,5,2
3,5,1,0,1312,12,524
4,5,1,0,1321,0,2461


In [8]:
acts["prs_nat"].unique()

array([1311, 1312, 1321, 1323, 1324, 1331, 1335, 1351, 1352, 1336, 1341,
       1318, 1316, 1322, 1361])

In [10]:
for nat in [1311, 1312, 1321, 1323, 1324, 1331, 1335, 1351, 1352, 1336, 1341, 1318, 1316, 1322, 1361]:
    count = np.sum(acts[acts['prs_nat']==nat]['act_dnb'])
    print(nat,count)

1311 815
1312 302150
1321 1375123
1323 788226
1324 3034672
1331 153469
1335 1017383
1351 6536863
1352 8191423
1336 622244
1341 36
1318 4196
1316 11037
1322 18274
1361 473


Du coup, on vire de la simulation les codes 1311 (ACTES EN K CHIRURGICAL), 1341 (ACTES D  ANATOMO-CYTO-PATHOLOGIE/MEDECINS), 1318 (PHONIATRE), 1361 (VIDEOCAPSULE) et même 1316 (diag en KE) pour obtenir une distribution d'actes (par region, age, sex du patient et specialité du médecin, etc)

In [11]:
acts=acts[ (acts["prs_nat"]!=1311) & (acts["prs_nat"]!=1341) & (acts["prs_nat"]!=1318) & (acts["prs_nat"]!=1361) & (acts["prs_nat"]!=1316) ]

On veut maintenant faire des correspondances possibles avec des codes CCAM, pour cela, on dispose dans un autre jeu de données (AMELI) qu'on va utiliser plus loin de code de regroupement des actes; on construit donc manuellement une correspondnace entre les codes de regroupement et les codes PRS_NAT.

Cette correspondance doit servir à préfiltrer les actes CCAM qui sont à associer lorsqu'on a 

In [13]:
prsnat_regrp= {1312:"ADC", #ACTES DE SPECIALITE EN K
               1321:"ADC", #ACTE DE CHIRURGIE CCAM
               1322:"ACO", #ACTE D'OBSTETRIQUE CCAM
               1323:"ADA", #ACTE D'ANESTHESIE CCAM
               1324:"ADE", #ACTE D'ECHOGRAPHIE CCAM
               1331:"ADI", #ACTES DE RADIOLOGIE
               1335:"ADI", #ACTE DE RADIOLOGIE MAMMOGRAPHIE
               1336:"ADI", #ACTE DE RADIOLOGIE MAMMOGRAPHIE DEPISTAGE
               1351:"ADI", #ACTE D'IMAGERIE (hors ECHOGRAPHIE) CCAM
               1352:"ADT"  #ACTES TECHNIQUES MEDICAUX  (hors IMAGERIE) CCAM
              }

Maintenant, tout comme pour les visites, on regarde combien on a d'actes en moyenne par personne sur une années ($\times 12$).

Pour cela, on commence par charger les infos sur la population ... et à l'aligner aux caractéristiques du jeu de l'OpenDamir

In [14]:
pop=pd.read_csv("pop.csv")
pop_rr=pop.groupby(['RR','sex','age']).agg({"pop":'sum'})
pop_rr.reset_index(inplace=True)
pop_rr.columns = pop_rr.columns.get_level_values(0)

## Define new Region (gather outre-mer lands and departements)
def new_rr(x):
    if x<=6: return 5
    else: return x
pop_rr['RR']=pop_rr['RR'].apply(new_rr)

##Define new age classes (compliant with DAMIR)
def new_age(x):
    x=x-x%10
    if x==10: x=0
    if x>80: x=80
    return x
pop_rr['age']=pop_rr['age'].apply(new_age)

# recompute the population
pop_rr=pop_rr.groupby(['RR','sex','age']).agg({"pop":'sum'})
pop_rr.reset_index(inplace=True)
pop_rr.columns = pop_rr.columns.get_level_values(0)

/home/tguyet/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,RR,sex,age,pop
0,1,1,0,10960.896988
1,1,1,5,13307.588357
2,1,1,10,14694.197617
3,1,1,15,14223.408864
4,1,1,20,10040.116261
5,1,1,25,8185.081731
6,1,1,30,7702.098690
7,1,1,35,8710.070308
8,1,1,40,11952.321274
9,1,1,45,13864.137616


In [17]:
## Mean number of acts per acts type per year per people (knowing their age, sex and region), whatever the specialist
acts_grp=acts.groupby(["RR","age","sex","prs_nat"]).agg({"act_dnb":"sum"}).reset_index()
nb_acts=pd.merge(acts_grp,pop_rr,on=["RR","age","sex"])
nb_acts['nb']=nb_acts['act_dnb']/nb_acts["pop"]*12 # *12 to have a yearly number
nb_acts=nb_acts[["RR","age","sex","prs_nat",'nb']]
nb_acts.head()

,RR,age,sex,prs_nat,nb
0,11,0,1,1312,0.006932
1,11,0,1,1321,0.532550
2,11,0,1,1323,0.202506
3,11,0,1,1324,0.306133
4,11,0,1,1331,0.184885


In [18]:
np.max(nb_acts['nb'])

7.927884083186598

In [21]:
nb_acts[ nb_acts['nb']>6 ]

,RR,age,sex,prs_nat,nb
138,11,80,1,1352,6.229433
147,11,80,2,1352,6.290893
858,44,70,1,1352,6.076103
867,44,70,2,1352,6.749799
1468,76,80,1,1352,6.443055
1477,76,80,2,1352,6.063973
1737,93,60,2,1352,6.362475
1746,93,70,1,1352,6.973111
1755,93,70,2,1352,6.918274
1764,93,80,1,1352,7.927884


In [22]:
nb_acts.to_csv("nb_acts_rragesex.csv")

## Spécialité du médecin exécutant

Maintenant, on aimerait avoir des indices sur la spécialité du médecin qui a réalisé un acte donné

On veut donc la proba d'avoir une specialité du médecin executant sachant l'acte et la personne traitée

$$p(S| RR,Sex,Age,PRS) = \frac{nb(S,RR,Sex,Age,PRS)}{nb(RR,Sex,Age,PRS)}$$

In [27]:
p_spe=pd.merge(acts,acts_grp, on=["RR","age","sex","prs_nat"])
p_spe['p']=p_spe['act_dnb_x']/p_spe['act_dnb_y']
p_spe=p_spe[["RR","age","sex","prs_nat","exe_spe",'p']]
p_spe

,RR,age,sex,exe_spe,prs_nat,p
0,5,0,1,1,1312,0.109983
1,5,0,1,5,1312,0.003384
2,5,0,1,12,1312,0.886633
3,5,0,1,0,1321,0.635753
4,5,0,1,1,1321,0.196073
...,...,...,...,...,...,...
18466,99,99,2,35,1324,0.500000
18467,99,99,2,6,1331,1.000000
18468,99,99,2,9,1352,0.067901
18469,99,99,2,31,1352,0.067901


In [28]:
p_spe.to_csv("p_exespe_acts.csv")

## Codes CCAM

Source: https://www.ameli.fr/l-assurance-maladie/statistiques-et-publications/donnees-statistiques/actes-techniques-de-la-ccam.php

On passe maintenant aux codes CCAM possibles ... pour cela, on va utiliser les données d'AMELI sur les proportions des actes CCAM.

On aimerait avoir la proba d'avoir un code CCAM, sachant le code PRSNAT

On a déjà les correspondances entre les PRS_NAT et les code de regroupement, on peut donc regarder la proba d'avoir un code CCAM, sachant un code regroupement.

In [59]:
actes = pd.read_excel("../data/Actes_techniques_de_la_CCAM_en_2016.xls", sheet_name='Panorama des actes CCAM')
actes.rename(columns={"Code Acte":'ccam',"Code Regroupement":"grp", "Quantité d'actes ":"nb"}, inplace=True)
actes=actes.groupby(['ccam',"grp"]).agg({"nb":"sum"}).reset_index()

On ne regarde quel les actes dans les groupements qui nous intéressent: 
* "ADC", #ACTES DE CHIRURGIE
* "ACO", #ACTE D'OBSTETRIQUE CCAM
* "ADA", #ACTE D'ANESTHESIE CCAM
* "ADE", #ACTE D'ECHOGRAPHIE CCAM
* "ADI", #ACTES DE RADIOLOGIE
* "ADT"  #ACTES TECHNIQUES MEDICAUX (hors IMAGERIE) CCAM

In [60]:
actes=actes[ (actes['grp']=="ADC") | (actes['grp']=="ACO") |(actes['grp']=="ADA") |(actes['grp']=="ADE") |(actes['grp']=="ADI") |(actes['grp']=="ADT") ]

En complément ... on vire également tous les codes relatifs aux dents (codes "HBXXXX"), cela élimine du coup:
* les actes de radiologie dentaire
* les actes de chirurgie dentaire (classés ADC) et qui étaient présents dans 24% des ADC!

In [61]:
actes=actes[ (actes['ccam'].str[:2]!="HB") ]

In [62]:
actes_tot=actes.groupby(['grp']).agg({"nb":"sum"}).reset_index()

In [63]:
p_act=pd.merge(actes,actes_tot,on=['grp'])
p_act['p']=p_act['nb_x']/p_act['nb_y']
p_act=p_act[['ccam',"grp",'p']]
p_act

,ccam,grp,p
0,AAFA001,ADA,1.101877e-05
1,AAFA002,ADA,4.525989e-05
2,AAFA003,ADA,1.066333e-06
3,AAFA005,ADA,9.478511e-07
4,AAFA006,ADA,2.369628e-07
...,...,...,...
9472,JQLF003,ACO,6.441742e-04
9473,JQQP001,ACO,1.923576e-04
9474,JQQP099,ACO,4.004617e-02
9475,JQQP900,ACO,0.000000e+00


In [64]:
np.max(p_act['p'])

0.36750140913116996

In [66]:
p_act[p_act['p']>0.05]

,ccam,grp,p
232,AHQJ021,ADA,0.057817
351,BFGA004,ADA,0.070904
4392,ZZLP025,ADA,0.203950
4815,BFGA004,ADC,0.080335
8705,QZFA036,ADC,0.058776
8836,DZQM006,ADE,0.126064
8901,PCQM001,ADE,0.070635
8902,QEQM001,ADE,0.097244
9361,YYYY600,ADI,0.192652
9366,ZBQK002,ADI,0.063450


In [72]:
corresp=pd.DataFrame.from_dict(prsnat_regrp, orient='index').reset_index()
corresp.rename(columns={"index":"prs_nat",0:'grp'},inplace=True)
corresp

,prs_nat,grp
0,1312,ADC
1,1321,ADC
2,1322,ACO
3,1323,ADA
4,1324,ADE
5,1331,ADI
6,1335,ADI
7,1336,ADI
8,1351,ADI
9,1352,ADT


In [78]:
p_act_prs=pd.merge(p_act,corresp,on="grp",how="outer")[["prs_nat","ccam","p"]]
p_act_prs

,prs_nat,ccam,p
0,1323,AAFA001,1.101877e-05
1,1323,AAFA002,4.525989e-05
2,1323,AAFA003,1.066333e-06
3,1323,AAFA005,9.478511e-07
4,1323,AAFA006,2.369628e-07
...,...,...,...
15320,1322,JQQP001,1.923576e-04
15321,1322,JQQP099,4.004617e-02
15322,1322,JQQP900,0.000000e+00
15323,1322,YYYY069,4.697104e-03


In [76]:
p_act_prs.to_csv("p_act_prs.csv")